# Lab 3

# C3.1 Реалiзуйте хеш функцiю SHA256.


In [15]:

'''Таблица констант
(первые 32 бита дробных частей кубических корней первых 64 простых чисел [от 2 до 311])'''
K256 = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2,
]

def ch(x: int, y: int, z: int):
    return (x & y) ^ (~x & z)

def majority(x: int, y: int, z: int):
    return (x & y) ^ (x & z) ^ (y & z)


def shift_right(x, n):
    return (x & 0xffffffff) >> n


def rotate_right(x, y):
    return (((x & 0xffffffff) >> (y & 31)) | (x << (32 - (y & 31)))) & 0xffffffff


def sigma0(x):
    return rotate_right(x, 2) ^ rotate_right(x, 13) ^ rotate_right(x, 22)


def sigma1(x):
    return rotate_right(x, 6) ^ rotate_right(x, 11) ^ rotate_right(x, 25)


def gamma0(x):
    return rotate_right(x, 7) ^ rotate_right(x, 18) ^ shift_right(x, 3)


def gamma1(x):
    return rotate_right(x, 17) ^ rotate_right(x, 19) ^ shift_right(x, 10)


def generate_hash(message):
    if isinstance(message, str):
        message = bytearray(message, 'ascii')
    elif isinstance(message, bytes):
        message = bytearray(message)
    else:
        print('Error')
        return None
        #(первые 32 бита дробных частей квадратных корней первых восьми простых чисел [от 2 до 19]):
    h0 = 0x6a09e667
    h1 = 0xbb67ae85
    h2 = 0x3c6ef372
    h3 = 0xa54ff53a
    h5 = 0x9b05688c
    h4 = 0x510e527f
    h6 = 0x1f83d9ab
    h7 = 0x5be0cd19
    length = len(message) * 8 # длинна это количество байтов
    message.append(0x80) # конкатенация с единичным битом
    while (len(message) * 8 + 64) % 512 != 0:
        message.append(0x00)# конкатенация с нулевыми битами

    message += length.to_bytes(8, 'big')#конкатенция с длинной исходного сообщения в битах в виде 64-битного числа с порядком байтов от старшего к младшему
    blocks = [message[i:i+64] for i in range(0, len(message), 64)] #разбиение на блоки для обработки
    for message_block in blocks:
        # Prepare message schedule
        message_schedule = []
        for t in range(0, 64):
            if t <= 15:
                message_schedule.append(bytes(message_block[t*4:(t*4)+4]))
            else:#генерируем дополнительные 48 слов 
                term1 = gamma1(int.from_bytes(message_schedule[t-2], 'big'))
                term2 = int.from_bytes(message_schedule[t-7], 'big')
                term3 = gamma0(int.from_bytes(message_schedule[t-15], 'big'))
                term4 = int.from_bytes(message_schedule[t-16], 'big')
                schedule = ((term1 + term2 + term3 + term4) % 2**32).to_bytes(4, 'big')
                message_schedule.append(schedule)

        # иницыализация доп переменных
        a = h0
        b = h1
        c = h2
        d = h3
        e = h4
        f = h5
        g = h6
        h = h7

        # основной блок
        for t in range(64):
            t1 = ((h + sigma1(e) + ch(e, f, g) + K256[t] + int.from_bytes(message_schedule[t], 'big')) % 2**32)
            t2 = (sigma0(a) + majority(a, b, c)) % 2**32
            h = g
            g = f
            f = e
            e = (d + t1) % 2**32
            d = c
            c = b
            b = a
            a = (t1 + t2) % 2**32

        # Добавить полученные значения к ранее вычисленному результату
        h0 = (h0 + a) % 2**32
        h1 = (h1 + b) % 2**32
        h2 = (h2 + c) % 2**32
        h3 = (h3 + d) % 2**32
        h4 = (h4 + e) % 2**32
        h5 = (h5 + f) % 2**32
        h6 = (h6 + g) % 2**32
        h7 = (h7 + h) % 2**32

    return ((h0).to_bytes(4, 'big') + (h1).to_bytes(4, 'big') +
            (h2).to_bytes(4, 'big') + (h3).to_bytes(4, 'big') +
            (h4).to_bytes(4, 'big') + (h5).to_bytes(4, 'big') +
            (h6).to_bytes(4, 'big') + (h7).to_bytes(4, 'big'))



In [17]:
print(generate_hash("AskaLengli").hex())

3042ad0c909b04af773ca27369680dc9fbbe3fed95fe8e5ea4fab5026e405f4e


# C3.2 Реалiзуйте алгоритм генерування секретного ключа криптосистеми AES-128 на основi хеш функцiї SHA256

In [27]:
len(generate_hash("AskaLengli").hex())

64

In [28]:
def generate_key(secret_key):#ключ в AES-128 массив 4х4 значений от 0 до 255
    lst = [int(generate_hash(secret_key).hex()[i:i+2], 16) for i in range(16)]
    #первые 32 значения хэш-функции используем как ключ 
    return [lst[i:i+4] for i in range(4)]
s_k = "AskaLengli"
generate_key(s_k)

[[48, 4, 66, 42], [4, 66, 42, 173], [66, 42, 173, 208], [42, 173, 208, 12]]

# C3.3 Реалiзуйте алгоритм створення i перевiрки HMAC на основi хеш функцiї SHA256

In [33]:
import hmac, hashlib
def HMAC1(key, msg):
    print(hmac.new(
        key=b'secret_shared_key',
        msg=open('message.txt', 'rb').read(),
        digestmod=hashlib.md5
    ).hexdigest())

In [35]:
def HMAC2( key, msg ) :# ПСЕВДОКОД
    # Если размер ключа больше, чем размер блока ...
    while len( key ) != block_size:
        if len( key ) > block_size:
            # Укорачиваем ключ до размера результата хеш-функции
            key = key[: block_size]
            # (Размер результата хеш-функции обычно меньше (а не равен), чем размер блока хеш-функции)
        # Если ключ меньше, чем размер блока хеш-функции ...
        if length( key ) < block_size:
            # Дополняем ключ нулевой последовательностью
            key = key + b'0'
            # оператор "∥" выполняет склейку строк (последовательностей байт)

    ipad = [ '\x36' * block_size ]
    # оператор "*" указывает количество повторений последовательности байт,
    # а block_size - размер блока хеш-функции, 
    opad = [ '\x5c' * block_size ]
    
    ikeypad = ipad ^ key
    # оператор "^" выполняет побитовое исключающее ИЛИ (xor)
    okeypad = opad ^ key
    
    return hash( okeypad + hash( ikeypad.append(msg) ) )
    # Оператор "+" выполняет склейку строк